In [2]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
import import_ipynb
#변수명 추출
from varname import nameof
#os정렬
import natsort
from tqdm import tqdm
# from standard_func import *
# import standard_func_py as sf
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 진행 여부 함수
def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[0] == 'y':
            return True
        if reply[0] == 'n':
            return False
        else:
            print("실패: 'y' 혹은 'n' 를 입력해 주세요")

### 함수

In [3]:
# blnk/검색 로직

def blnk_chk(data):
    cnt_chk = len(data)

    chk_nll_list = []
    chk_nll_cnt_list = []
    # null값 체크(개수 파악)
    for i,j in zip(range(len(data.columns)), data.columns):
        indv_cnt = data.loc[data[j] != '', j]
        if len(data[j]) != len(indv_cnt):
            print(j + ": " + str(len(indv_cnt)))
            chk_nll_list.append(j)
            chk_nll_cnt_list.append(len(indv_cnt) - cnt_chk)
    print("================================")
    if len(chk_nll_list) == 0:
        print("모든 값이 입력되었습니다.")
    elif len(chk_nll_list) > 1 :
        print("blnk(null)이 포함된 컬럼이 탐지되었습니다.")
        print(chk_nll_list)
        print("================================")

    globals()["cnt_chk_out"] = pd.DataFrame(zip(chk_nll_list, chk_nll_cnt_list), columns = ["NULL포함컬럼명", "NULL개수"])
    print(cnt_chk_out)
    
    print("================================")
    print("null 인덱스를 임시 보관합니다.")
    print("================================")
    nll_chk_row = cnt_chk_out.iloc[:,0].to_list()
    for i, j in zip(range(len(nll_chk_row)), nll_chk_row):
        globals()["null_{}".format(i)] = data.loc[data[j] == '', j].index
        print("널인덱스보관_" + str(j) + "(변수명) -> null_{}".format(i))
        
# 스페이스 제거 함수
def rmv_spce(data, opt, *col):
    if opt == 'all': 
        for i in data.columns:
            data[i] = data[i].str.replace(" ", "")
            print(str(i) + ":\t 스페이스 제거 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].str.replace(" ", "")
        print(col + ":\t 스페이스 제거 성공")
#         print(type(col))

# 불필요 컬럼 빼기 - 이름으로 빼기
def brief_col(data, excpt_col_list, name):
    col_list = data.columns
    selected_col_list = [x for x in col_list if x not in excpt_col_list]
    print(selected_col_list)
    globals()["{}".format(name)] = data[selected_col_list]
    
# 불필요 컬럼 빼기 - 순서로 빼기 
# tmp_col_list = [5,6,7,8]
# tmp_tbl_list = [2,3,4,5]
def brief_col_num(data, excpt_col_list, name):
    col_list = data.columns
    col_list_num = [*range(len(col_list))]
    selected_col_list_num = [x for x in col_list_num if x not in excpt_col_list]
    selected_col = col_list[selected_col_list_num]
    print(selected_col)
    globals()["{}".format(name)] = data[selected_col]

# 컬럼명(순서) 선택 및 변경
def cor_col(data, select_col, modify_col, name):
    # 컬럼명(순서) 선택
    globals()["{}".format(name)] = data[select_col]
    # 컬렴명 변경
    globals()["{}".format(name)].columns = modify_col
    
# null 공백처리 함수
def nll_to_blnk(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 Null 공백 처리]")
        for i in data.columns:
            data[i] = data[i].fillna("")
            print(str(i) + ":\t Null 공백 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].fillna("")
        print(col + ":\t  공백 처리 성공")

# 컬럼 고유값 수 비교
def cmpar(data, col1, col2):
    crtr_1 = len(data[col1].unique()) - 1
    crtr_2 = len(data[col2].unique()) - 1
    if crtr_1 == crtr_2:
        print('[정상] 용어명과 분리된용어명의 고유값 수가 같습니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
    else:
        print('[오류] 용어명과 분리된용어명의 고유값 수가 다릅니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
        
# 공백 null처리 함수
def blank_to_nll(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 공백 Null 처리]")
        for i in data.columns:
            data.loc[data[i] == '', i] = None
            print(str(i) + ":\t Null 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data.loc[data[col] == '', col] = None
        print(col + ":\t  Null 처리 성공")
        
def row_nll_rmv(data, name):
    ### 값이 한개라도 있을 경우만 카운트 ###

    data_cnt = len(data)
    notna_list = []
    for i in range(data_cnt):    
        # row 기준 널 체크(값이 한개라도 있을 경우 포함)
        if data.iloc[i].notna().sum() > 1:
            notna_list.append(i)
    globals()["{}".format(name)] = globals()["{}".format(data)].loc[notna_list]
    print(notna_list)
    print(data.shape)
    
# blank row 삭제
def blnk_row_rmv(data, opt, *col):
    if opt == 'all':
        data_cnt = len(data)
        notna_list = []
        for i in range(data_cnt):
            if (data.iloc[i,:] != '').sum() > 1:
                notna_list.append(i)
        globals()["blnk_rmv_all".format(col)] = data.iloc[notna_list]
        print(globals()["blnk_rmv_all".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
    elif opt == 'col':
        data_cnt = len(data)
        notna_list = []
        col = ''.join(col)
        for i in range(data_cnt):
            if (data.loc[i,col] != ''):
                notna_list.append(i)
        globals()["blnk_rmv_{}".format(col)] = data.iloc[notna_list].copy()
        print(globals()["blnk_rmv_{}".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")

In [4]:
# 파이썬 로드후 파일 실행
def read_grip_file(data, path, file_name):
    os.system(path + file_name + ".xlsx")
    globals()[file_name] = pd.read_excel(path + file_name + ".xlsx", index_col = 0, na_filter=False)


# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    print('예) path + "\\metadata\\" + folder + "\\01 abnormal_words_correct", "abnormal_words_correct"')
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

### 추가 1) 메타데이터 띄움처리 전 중복 제거..
### 추가 2) 한글명 notnull vs null 분리하여 notnull만 관리(null: 파일 생성 및 기입 요청)
### 추가 3.1) 한글 용어 점검 기능: 한글명+영문명+숫자조합 가능), 그외 모두 제거
### 추가 3.2) 한글 단어 점검 기능: 한글명(숫자조합 가능), 영문명만(숫자조합 가능) 그외 모두 제거
### 추가 4) 테이블기준 상세항목 정의하기.. 

In [5]:
# 1.전체 데이터 로드

In [6]:
# 경로 및 파일명 설정
path = os.path.join(os.getcwd())
path

'C:\\Users\\namdd\\OneDrive\\Data\\Standardization'

In [7]:
# 시스템 리스트업
folder_list = pd.read_excel(path + "\\metadata\\철도공사-업무별-시스템 담당자_표준화용.xlsx", header=1, sheet_name="표준화대상_데이터관리")['폴더관리1']
folder_list = folder_list.map(lambda x : x.split('md ')[-1])
# folder_list = folder_list[folder_list != "00_"].reset_index(drop=True)
folder_list = list(folder_list.unique())
folder = folder_list[29]
folder

'30_광역철도수입관리시스템'

In [ ]:
# 모든 테이블 로드
for i in folder_list:
    print(i)
    globals()["data_{}".format(i)] = pd.read_csv(path + "\\metadata\\" + i + "\\METADATA_UTF8.csv", na_filter = False)
#     globals()["data_{}".format(i)] = pd.DataFrame()

### 예제)data_30_광역철도수입관리시스템

In [ ]:
input_meta = data_30_광역철도수입관리시스템

# 데이터 소수점 전처리
input_meta.loc[input_meta['DATA_PRECISION'] != ''] 
input_meta.loc[input_meta['DATA_PRECISION'] != '', 'DATA_LENGTH'] =  input_meta['DATA_PRECISION']
# 컬럼명 정리
col_list = ['관리부서코드', '관리부서명', '시스템명', '물리DB명', 'TABLE_NAME', 'COMMENTS',
       'COLUMN_NAME', '컬럼물리명FullName', 'COMMENTS.1', '컬럼설명', '표준도메인그룹명',
       '도메인분류명', 'DATA_TYPE', 'DATA_LENGTH', 'DATA_SCALE']
input_meta = input_meta[col_list]
smpl_col = ["관리부서코드*", "관리부서명*", "시스템명*", "물리DB명*", "테이블물리명*", "테이블한글명*", "컬럼물리명*", "컬럼물리명FullName",
 "컬럼한글명*", "컬럼설명", "표준도메인그룹명*", "도메인분류명*", "데이터타입*", "데이터길이*", "소수점자리"]
input_meta.columns = smpl_col
input_meta = input_meta.fillna('')
input_meta.head()

In [ ]:
# 2.중복제거 후 형태이상단어 검색

In [ ]:
input_meta.groupby('컬럼한글명*').head(1)